In [1]:
import keras
import pandas
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import SGD
import numpy as np
from time import time
from collections import Counter
import datetime
from keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
kdd_data = pandas.read_csv("kddcup.data_10_percent\\kddcup.data_10_percent_corrected", header=None, names = col_names)
kdd_data.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494021.000000,4.940210e+05,4.940210e+05,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,...,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000,494021.000000
mean,47.979302,3.025610e+03,8.685324e+02,0.000045,0.006433,0.000014,0.034519,0.000152,0.148247,0.010212,...,232.470778,188.665670,0.753780,0.030906,0.601935,0.006684,0.176754,0.176443,0.058118,0.057412
std,707.746472,9.882181e+05,3.304000e+04,0.006673,0.134805,0.005510,0.782103,0.015520,0.355345,1.798326,...,64.745380,106.040437,0.410781,0.109259,0.481309,0.042133,0.380593,0.380919,0.230590,0.230140
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
kdd_data = kdd_data.drop(['protocol_type','service','flag','num_outbound_cmds','is_host_login','label'], axis=1)

In [4]:
kdd_data = kdd_data/kdd_data.max()
var = kdd_data.std()
var = var.sort_values(ascending = False)
print(var)

srv_count                      0.482041
dst_host_same_src_port_rate    0.481309
count                          0.417118
dst_host_srv_count             0.415845
dst_host_same_srv_rate         0.410781
same_srv_rate                  0.388189
srv_serror_rate                0.381017
dst_host_srv_serror_rate       0.380919
serror_rate                    0.380717
dst_host_serror_rate           0.380593
logged_in                      0.355345
dst_host_count                 0.253903
srv_rerror_rate                0.232147
rerror_rate                    0.231623
dst_host_rerror_rate           0.230590
dst_host_srv_rerror_rate       0.230140
srv_diff_host_rate             0.142397
dst_host_diff_srv_rate         0.109259
diff_srv_rate                  0.082205
wrong_fragment                 0.044935
dst_host_srv_diff_host_rate    0.042133
is_guest_login                 0.037211
hot                            0.026070
duration                       0.012134
root_shell                     0.010551


In [5]:
corelation=kdd_data
c = corelation.corr().abs()
c = c.sum()
c = c.sort_values(ascending = True)
print(c)

src_bytes                       1.042325
land                            1.243656
urgent                          1.307546
num_shells                      1.449271
wrong_fragment                  1.544118
dst_bytes                       1.554732
num_file_creations              1.566782
num_failed_logins               1.594131
is_guest_login                  2.395053
hot                             2.458057
duration                        2.506596
root_shell                      2.518237
num_access_files                2.786512
su_attempted                    3.357793
srv_diff_host_rate              3.466152
num_root                        3.556537
num_compromised                 3.563601
dst_host_srv_diff_host_rate     3.589503
dst_host_count                  5.410326
dst_host_diff_srv_rate          5.578771
diff_srv_rate                   5.740627
logged_in                       5.910130
srv_rerror_rate                 7.029334
dst_host_srv_rerror_rate        7.030876
rerror_rate     

In [9]:
v = var.iloc[0:13].axes
c1 = c.iloc[0:13].axes
list1 = set(list(v[0]) + list(c1[0]))
print(list1, len(list1))

{'serror_rate', 'dst_bytes', 'num_file_creations', 'duration', 'src_bytes', 'srv_rerror_rate', 'dst_host_srv_count', 'is_guest_login', 'root_shell', 'num_failed_logins', 'land', 'hot', 'dst_host_serror_rate', 'dst_host_same_srv_rate', 'srv_serror_rate', 'dst_host_srv_serror_rate', 'wrong_fragment', 'num_access_files', 'logged_in', 'dst_host_count', 'dst_host_same_src_port_rate', 'count', 'num_shells', 'urgent', 'same_srv_rate', 'srv_count'} 26
